## 한글 폰트 설치

- 참고: https://teddylee777.github.io/colab/colab-korean

In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## 데이터셋

In [ ]:
from teddynote import dataset

dataset.download('지하철승하차인원')

- [서울교통공사_월별 승하차인원](https://www.data.go.kr/data/15044249/fileData.do)
- [지하철역 주소 정보](http://data.seoul.go.kr/dataList/OA-12035/S/1/datasetView.do)

## 모듈 import & 파일 로드

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'NanumGothic'

DATA = 'data'

In [ ]:
# 파일 로드
file1 = os.path.join(DATA, '지하철_승하차인원.csv')
df = pd.read_csv(file1, encoding='euc-kr')
df

## 역명에 대한 클린징

In [ ]:
import re

# Regular Expression 적용하여 ()괄호를 제거 후 추출
pattern_string = r'.+(?=\()'
pattern = re.compile(pattern_string)

In [ ]:
pattern.match('올림픽공원(9)').group().strip()

In [ ]:
def clean_station(x):
    # 코드입력
    

In [ ]:
# 코드입력
df['역명_clean'] = 
df[['역명_clean', '역명']].head(10)

`역명` 컬럼을 제거한 뒤, `역명_clean`컬럼을 `역명`으로 변환합니다.

In [ ]:
# 코드입력
df = 
df = 
df.head()

## 중복 되는 역에 대한 승하차 인원 합계 환산

In [ ]:
df.loc[df['역명'] == '서울역']

In [ ]:
df.loc[df['역명'] == '고속터미널']

`역명`을 기준으로 승하차인원에 대한 합계를 구합니다.

In [ ]:
# 코드입력
df_total = 
df_total

**역번호**도 합계가 구해졌기 때문에, 역번호를 이전 데이터프레임으로부터 병합합니다.

In [ ]:
df_merged = pd.merge(df_total, df.drop_duplicates('역명', keep='first')[['역명', '역번호']], how='left')
df_merged

**고속터미널** 역에 대한 합계치 비교

In [ ]:
df.loc[df['역명'] == '고속터미널']

In [ ]:
df_merged.loc[df_merged['역명'] == '고속터미널']

## `seoul_station_geocode.csv` 파일 로드

In [ ]:
# 코드입력
address = pd.read_csv('seoul_station_geocode.csv')
# csv 파일이 없는 경우
# address = pd.read_csv('https://raw.githubusercontent.com/teddylee777/dip/main/assets/seoul_station_geocode.csv')
address.head()

In [ ]:
# 코드입력
merged_data = 
merged_data.head()

In [ ]:
df_merged.shape, merged_data.shape

병합된 데이터셋에 대한 결측치 확인

In [ ]:
merged_data.isnull().sum()

## 주소 데이터 분할

In [ ]:
# 코드입력


**구**만 추출하기

In [ ]:
# 코드입력


In [ ]:
# 코드입력
merged_data['구'] = 
merged_data['구']

In [ ]:
merged_data.head()

## folium을 활용한 지도 시각화

In [ ]:
import folium

# 위도
latitude = 37.394946
# 경도
longitude = 127.111104

m = folium.Map(location=[latitude, longitude],
               zoom_start=17, 
               width=750, 
               height=500
              )
m

In [ ]:
station_name = '시청'
geo = list(merged_data.loc[merged_data['역명'] == station_name, ['lat', 'lng']].iloc[0])
print(f'위도: {geo[0]}, 경도: {geo[1]}')

# folium 시각화
m = folium.Map(location=geo,
               zoom_start=17, 
               width=750, 
               height=500
              )

folium.Marker(geo,
              popup=station_name,
              tooltip=f'{station_name} 정보').add_to(m)
m

타일을 `cartodbpositron`으로 변경합니다.

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)
m

## 서울 행정구역 표기 및 시각화에 boundary 적용

In [ ]:
# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)
seoul_geo

자치구별 1월의 승하차인원 합계량을 산출합니다.

In [ ]:
# 코드입력


In [ ]:
# 월 설정
N = 2

plt.figure(figsize=(12, 15))
merged_data.groupby('구')[f'{N}월'].sum().sort_values().plot(kind='barh')
plt.title(f'{N}월 구별 승하차 인원')
plt.show()

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=10, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m.choropleth(geo_data=seoul_geo,
             data=merged_data.groupby('구')[f'{N}월'].sum(),
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.9,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name=f'{N}월 승하차 인원', 
            )
m

## 통계 분석

In [ ]:
stats = merged_data.drop(['주소', '역번호', 'lat', 'lng'], axis=1)
stats

In [ ]:
stats.loc[stats['역명'] == '강남']

In [ ]:
stats.loc[stats['역명'] == '강남'].iloc[:, 1:-1]

[seaborn pointplot](https://seaborn.pydata.org/generated/seaborn.pointplot.html)

In [ ]:
place = '강남'
plt.figure(figsize=(10, 6))
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(stats.loc[stats['역명'] == place].iloc[:, 1:-1].values))
plt.show()

In [ ]:
place1 = '강남'
place2 = '고속터미널'
plt.figure(figsize=(10, 6))
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(stats.loc[stats['역명'] == place1].iloc[:, 1:-1].values), color='black')
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(stats.loc[stats['역명'] == place2].iloc[:, 1:-1].values), color='tomato')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.pointplot(x ='역명', y='1월', data=stats.loc[stats['구'] == '강남구'].iloc[:, :-1], color='black')
sns.pointplot(x ='역명', y='4월', data=stats.loc[stats['구'] == '강남구'].iloc[:, :-1], color='blue')
sns.pointplot(x ='역명', y='7월', data=stats.loc[stats['구'] == '강남구'].iloc[:, :-1], color='tomato')
sns.pointplot(x ='역명', y='11월', data=stats.loc[stats['구'] == '강남구'].iloc[:, :-1], color='cyan')
plt.show()

In [ ]:
stats.loc[stats['구'] == '강남구'].iloc[:, :-1]

강남구에 위치한 지하철역의 승하차인원을 heatmap으로 시각화합니다.

In [ ]:
plt.figure(figsize=(12, 12))
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
sns.heatmap(stats.loc[stats['구'] == '강남구'].iloc[:, 1:-1]/1000, annot=True, fmt='.1f', cmap=cmap, cbar_kws={"shrink": .7})
plt.yticks(np.arange(len(stats.loc[stats['구'] == '강남구'].iloc[:, 0].values)), stats.loc[stats['구'] == '강남구'].iloc[:, 0].values)
plt.yticks(rotation=45)
plt.show()

강남구에 위치한 역의 승하차 인원의 평균을 산출하여 `avg` 임시 변수에 대입합니다.

In [ ]:
avg = stats.loc[stats['구'] == '강남구'].iloc[:, 1:-1].mean()
avg

In [ ]:
place1 = '강남'
plt.figure(figsize=(10, 6))
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(stats.loc[stats['역명'] == place1].iloc[:, 1:-1].values), color='black')
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(avg.values), color='tomato')
plt.show()

In [ ]:
gangnam = stats.loc[stats['역명'] == place1].iloc[:, 1:-1].values
gangnam

강남역의 승하차 인원에 대한 정규화를 수행합니다.

In [ ]:
gangnam_norm = (gangnam - gangnam.min()) / (gangnam.max() - gangnam.min())
gangnam_norm

In [ ]:
avg = avg.values
avg

강남구 평균 승하차 인원에 대한 정규화를 수행합니다.

In [ ]:
avg_norm = (avg - avg.min()) / (avg.max() - avg.min())
avg_norm

In [ ]:
place1 = '강남'
plt.figure(figsize=(10, 6))
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(gangnam_norm), color='black')
sns.pointplot(x = np.arange(1, 13), y=np.squeeze(avg_norm), color='tomato')
plt.show()